In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

import pydicom
from pydicom.data import get_testdata_files

In [ ]:
train=pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test=pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
train

In [ ]:
test.head()

In [ ]:
smokers=train.loc[train.SmokingStatus=='Currently smokes']

In [ ]:
train.info()

In [ ]:
sns.pairplot(train)

In [ ]:
sns.pairplot(train,hue='Sex')

The distribution of FVC of males is shifted to higher FVC values indicating males have a higher FVC on average

In [ ]:
sns.pairplot(train,hue='SmokingStatus')

Age distribution of currently smoking patients has multiple small peaks across all ages and, a single high peak near age 70(like other two categories). This indicates pulmonary fibrosis affects middle aged people who are currently smokers.

In [ ]:
sns.distplot(smokers['Age'])

In [ ]:
sns.distplot(train['Age'])

In [ ]:

df= train.groupby([train.Patient,train.Age,train.Sex, train.SmokingStatus])['Patient'].count()
df.index = df.index.set_names(['id','Age','Sex','SmokingStatus'])
df = df.reset_index()
df.rename(columns = {'Patient': 'freq'},inplace = True)
print(df.shape)

In [ ]:
fig = px.bar(df, x='freq',y ='id',color='freq')
fig.update_layout(yaxis={'categoryorder':'total ascending'},title='No. of observations for each patient')
fig.update_yaxes(showticklabels=False)
fig.show()

In [ ]:
plt.figure(figsize=[10,8])
plt.style.use('ggplot')
sns.countplot(x='Sex',data=train,hue='SmokingStatus')

In [ ]:
train.corr()

In [ ]:
sns.heatmap(train.corr(),annot=True)

In [ ]:
fig = px.line(train.loc[train['Patient']=='ID00422637202311677017371'],x='Weeks',y='FVC')
fig.show()

In [ ]:
fig = px.line(train.loc[train['Patient']=='ID00426637202313170790466'],x='Weeks',y='FVC')
fig.show()

In [ ]:
p1=train.loc[train['Patient']=='ID00426637202313170790466']
p2=train.loc[train['Patient']=='ID00007637202177411956430']
p3=train.loc[train['Patient']=='ID00355637202295106567614']
pati=pd.concat([p1,p2,p3])
px.line(pati,x='Weeks',y='FVC',line_group='Patient',color='Patient')





In [ ]:
PathDicom = '/kaggle/input/osic-pulmonary-fibrosis-progression/'
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))

In [ ]:
print(lstFilesDCM[0])

In [ ]:
RefDs = pydicom.dcmread(lstFilesDCM[4])
print(RefDs)

In [ ]:
print('Patient id is {}'.format(RefDs.PatientID))
print('Sex..........{}'.format(RefDs.PatientSex))
print('Image Position {}'.format(RefDs.ImagePositionPatient))
print('Image Orientation {}'.format(RefDs.ImageOrientationPatient))

In [ ]:
def MakeDF(lst):
    dictDf={}
    refd=pydicom.dcmread(lst[0])
    dictDf['Patient']=[refd.PatientID]
    dictDf['rows']=[refd.Rows]
    for i in range(1,len(lst)):
        refd=pydicom.dcmread(lst[i])
        dictDf['Patient'].append(refd.PatientID)
        dictDf['rows'].append(refd.Rows)
        #print(dictDf)
    return(dictDf)    

In [ ]:
pd_dict=MakeDF(lstFilesDCM)

In [ ]:
df=pd.DataFrame.from_dict(pd_dict)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(RefDs.pixel_array, cmap=plt.cm.bone)
plt.show()

approch is to use autoencoder to encode image data and concat it with tabular data and then use rnn for predicting time series data